In [39]:
import pandas as pd
import numpy as np
import matplotlib.ploty as ply

ModuleNotFoundError: No module named 'matplotlib'

In [15]:
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import load_img

In [4]:
path ='./clothing-dataset-small/train/t-shirt/'
file = '1cab0eb2-60f6-4d0a-a380-9a07b834d855.jpg'
fullName = f'{path}/{file}'

In [22]:
img = load_img(fullName, target_size=(299,299))

In [9]:
print(img)

<PIL.Image.Image image mode=RGB size=150x150 at 0x1A3E6E429A0>


In [26]:
x = np.array(img)
x.shape

(299, 299, 3)

In [37]:
from keras.applications.xception import Xception, preprocess_input, decode_predictions

In [29]:
model = Xception(weights='imagenet', input_shape=(299,299,3))

In [32]:
X = np.array([x])
X = preprocess_input(X)

In [35]:
pred = model.predict(X)
pred.shape

(1, 1000)

In [38]:
decode_predictions(pred)

49152/35363 [=========================================] - 0s 0us/step


[[('n03595614', 'jersey', 0.47058782),
  ('n04033995', 'quilt', 0.021494234),
  ('n04370456', 'sweatshirt', 0.016343094),
  ('n03188531', 'diaper', 0.016156502),
  ('n02730930', 'apron', 0.013485141)]]